In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
import web_scraping as ws

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

In [162]:
with open('combined_movie_df_3.pickle', 'rb') as read_file:
    movie_df = pickle.load(read_file)

In [231]:
with open('combined_movie_df_4.pickle', 'wb') as to_write:
    pickle.dump(movie_df, to_write)

In [106]:
# Put in data_cleaning module

def find_unique_genres(df):
    unique_genres = []
    genres_list_of_lists = df['genres'].value_counts().index.to_list()
    for genres_list in genres_list_of_lists:
        unique_genres += genres_list
#     unique_genres_set = set(unique_genres)
    return set(unique_genres)

# def clean_unique_genres_set(unique_genres_set):
#     cleaned_set = set()
#     for genre in unique_genres_set:
#         cleaned_set.add(genre.strip())
#     return cleaned_set

In [130]:
# Put in feature_engineering module

def create_genre_encodings(df, unique_genres):
    for genre in unique_genres:
        df['is_' + genre] = df.apply(lambda x: genre in x['genres'], axis=1).astype(int)

In [120]:
is_genre_cols = ['is_Action', 'is_Adventure',
                 'is_Animation', 'is_Biography', 'is_Comedy', 'is_Crime', 'is_Drama',
                 'is_Family', 'is_Fantasy', 'is_History', 'is_Horror', 'is_Music',
                 'is_Musical', 'is_Mystery', 'is_News', 'is_Romance', 'is_Sci-Fi',
                 'is_Sport', 'is_Thriller', 'is_War', 'is_Western']

In [153]:
movie_df['usa_release_date'].str.replace(r'.\((.+)\)', '').tail(20)

2826    26 February 1941
2827        20 June 1941
2828    27 December 1940
2829     6 December 1940
2830        21 July 1941
2831       31 March 1940
2832    23 February 1940
2833    22 December 1939
2834        9 March 1938
2835     24 January 1938
2836     4 February 1938
2837         1 June 1934
2838     18 October 1932
2839       20 April 1930
2840     8 February 1930
2841      1 October 1925
2842     1 February 1925
2843       December 1923
2844      September 1923
2845      1 January 1920
Name: usa_release_date, dtype: object

In [216]:
movie_df['other_release_date'] = movie_df['other_release_date'].str.replace(r'.\((.+)\)', '')

In [218]:
movie_df['usa_release_date'] = pd.to_datetime(movie_df['usa_release_date'])

In [219]:
movie_df['japan_release_date'] = pd.to_datetime(movie_df['japan_release_date'])

In [220]:
movie_df['other_release_date'] = pd.to_datetime(movie_df['other_release_date'])

In [225]:
cols = movie_df.columns.to_list()

In [228]:
movie_df = movie_df[rearranged_cols]

In [229]:
movie_df.head()

,title,country,runtime_minutes,budget,global_gross,mpaa_rating,japan_release_date,usa_release_date,other_release_date,other_release_date_country,genres,imdb_user_rating,imdb_user_rating_count,oscar_wins,non_oscar_wins,metascore,is_Action,is_Adventure,is_Animation,is_Biography,is_Comedy,is_Crime,is_Drama,is_Family,is_Fantasy,is_History,is_Horror,is_Music,is_Musical,is_Mystery,is_News,is_Romance,is_Sci-Fi,is_Sport,is_Thriller,is_War,is_Western
0,A Whisker Away,Japan,104.0,NaN,NaN,TV-PG,2020-06-18,NaT,NaT,None,"[Animation, Adventure, Family, Fantasy, Romance]",6.8,2204.0,NaN,NaN,NaN,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0
1,True North,Japan,93.0,NaN,NaN,None,NaT,NaT,NaT,None,[Animation],6.5,24.0,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Psycho Pass 3: First Inspector,Japan,135.0,NaN,7.57e+05,None,2020-03-27,NaT,NaT,None,"[Animation, Sci-Fi]",7.6,187.0,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Altered Carbon: Resleeved,"Japan, USA",74.0,NaN,NaN,TV-MA,NaT,NaT,NaT,None,"[Animation, Action, Crime, Drama, Mystery, Sci...",6.5,3898.0,NaN,NaN,NaN,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0
4,Gekijôban Shirobako,Japan,119.0,NaN,2.59e+06,None,2020-02-29,NaT,NaT,None,"[Animation, Comedy, Drama]",6.4,16.0,NaN,NaN,NaN,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [227]:
rearranged_cols = ['title',
                   'country',
                   'runtime_minutes',
                   'budget',
                   'global_gross',
                   'mpaa_rating',
                   'japan_release_date',
                   'usa_release_date',
                   'other_release_date',
                   'other_release_date_country',
                   'genres',
                   'imdb_user_rating',
                   'imdb_user_rating_count',
                   'oscar_wins',
                   'non_oscar_wins',
                   'metascore',
                   'is_Action',
                   'is_Adventure',
                   'is_Animation',
                   'is_Biography',
                   'is_Comedy',
                   'is_Crime',
                   'is_Drama',
                   'is_Family',
                   'is_Fantasy',
                   'is_History',
                   'is_Horror',
                   'is_Music',
                   'is_Musical',
                   'is_Mystery',
                   'is_News',
                   'is_Romance',
                   'is_Sci-Fi',
                   'is_Sport',
                   'is_Thriller',
                   'is_War',
                   'is_Western']

In [177]:
movie_df['usa_release_date'].str.contains(r'\((.+)\)').tail(20)

2826    False
2827    False
2828    False
2829    False
2830     True
2831    False
2832    False
2833    False
2834     True
2835    False
2836    False
2837    False
2838    False
2839    False
2840    False
2841    False
2842    False
2843    False
2844    False
2845    False
Name: usa_release_date, dtype: object

In [187]:
movie_df['other_release_date'] = np.where(movie_df['usa_release_date'].str.contains(r'\((.+)\)'), movie_df['usa_release_date'], None)

/Users/binhhoang/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [199]:
my_list = [1, 2, 3]
my_list[-1]

3

In [213]:
movie_df['other_release_date_country'] = np.where(movie_df['other_release_date_country'] != None, movie_df['other_release_date_country'].str.replace(')', ''), None)

In [230]:
movie_df[movie_df['other_release_date_country'].notnull()].head()

,title,country,runtime_minutes,budget,global_gross,mpaa_rating,japan_release_date,usa_release_date,other_release_date,other_release_date_country,genres,imdb_user_rating,imdb_user_rating_count,oscar_wins,non_oscar_wins,metascore,is_Action,is_Adventure,is_Animation,is_Biography,is_Comedy,is_Crime,is_Drama,is_Family,is_Fantasy,is_History,is_Horror,is_Music,is_Musical,is_Mystery,is_News,is_Romance,is_Sci-Fi,is_Sport,is_Thriller,is_War,is_Western
1414,The Academy of Magic,USA,86.0,1.20e+07,NaN,None,NaT,2020-01-05,2020-01-05,Lebanon,"[Animation, Fantasy]",3.4,72.0,NaN,NaN,NaN,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1471,Thomas & Friends: Digs & Discoveries,USA,66.0,NaN,NaN,None,NaT,2019-09-11,2019-09-11,Netherlands,"[Animation, Family]",5.0,6.0,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1487,Trouble,USA,87.0,NaN,1.08e+07,None,NaT,2019-08-08,2019-08-08,Thailand,"[Animation, Comedy, Family]",6.1,931.0,NaN,NaN,NaN,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1498,Outlander,USA,NaN,3.47e+00,NaN,None,NaT,2019-07-04,2019-07-04,Australia,[Animation],NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1505,Norm of the North: King Sized Adventure,USA,90.0,NaN,1.44e+06,None,NaT,2019-08-02,2019-08-02,Vietnam,"[Animation, Adventure, Comedy, Family]",3.1,278.0,NaN,NaN,NaN,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [205]:
movie_df['other_release_date_country'] = np.where(movie_df['other_release_date_country'] != None, movie_df['other_release_date'][-1], None)

KeyError: -1

In [195]:
movie_df[movie_df['other_release_date_country'].notnull()]

,title,country,runtime_minutes,budget,global_gross,mpaa_rating,japan_release_date,usa_release_date,genres,imdb_user_rating,imdb_user_rating_count,oscar_wins,non_oscar_wins,metascore,is_Action,is_Adventure,is_Animation,is_Biography,is_Comedy,is_Crime,is_Drama,is_Family,is_Fantasy,is_History,is_Horror,is_Music,is_Musical,is_Mystery,is_News,is_Romance,is_Sci-Fi,is_Sport,is_Thriller,is_War,is_Western,other_release_date,other_release_date_country
1414,The Academy of Magic,USA,86.0,1.20e+07,NaN,None,NaT,5 January 2020 (Lebanon),"[Animation, Fantasy]",3.4,72.0,NaN,NaN,NaN,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5 January 2020 (Lebanon),"[5, January, 2020, (Lebanon)]"
1471,Thomas & Friends: Digs & Discoveries,USA,66.0,NaN,NaN,None,NaT,11 September 2019 (Netherlands),"[Animation, Family]",5.0,6.0,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,11 September 2019 (Netherlands),"[11, September, 2019, (Netherlands)]"
1487,Trouble,USA,87.0,NaN,1.08e+07,None,NaT,8 August 2019 (Thailand),"[Animation, Comedy, Family]",6.1,931.0,NaN,NaN,NaN,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8 August 2019 (Thailand),"[8, August, 2019, (Thailand)]"
1498,Outlander,USA,NaN,3.47e+00,NaN,None,NaT,4 July 2019 (Australia),[Animation],NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4 July 2019 (Australia),"[4, July, 2019, (Australia)]"
1505,Norm of the North: King Sized Adventure,USA,90.0,NaN,1.44e+06,None,NaT,2 August 2019 (Vietnam),"[Animation, Adventure, Comedy, Family]",3.1,278.0,NaN,NaN,NaN,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2 August 2019 (Vietnam),"[2, August, 2019, (Vietnam)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2786,The Wonderful World of the Brothers Grimm,USA,135.0,6.25e+06,NaN,G,None,16 July 1963 (UK),"[Animation, Adventure, Biography, Comedy, Dram...",6.4,1420.0,1.0,1.0,NaN,0,1,1,1,1,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,16 July 1963 (UK),"[16, July, 1963, (UK)]"
2788,The World of Topo Gigio,USA,75.0,NaN,NaN,None,None,1961 (Italy),"[Animation, Family, Fantasy]",5.6,40.0,NaN,1.0,NaN,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1961 (Italy),"[1961, (Italy)]"
2790,Donald Duck and his Companions,USA,NaN,NaN,NaN,None,None,13 September 1968 (Italy),"[Family, Animation, Comedy]",7.0,186.0,NaN,NaN,NaN,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,13 September 1968 (Italy),"[13, September, 1968, (Italy)]"
2830,Fantasia,USA,125.0,2.28e+06,7.64e+07,G,None,21 July 1941 (UK),"[Animation, Family, Fantasy, Music]",7.8,86287.0,NaN,8.0,96.0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,21 July 1941 (UK),"[21, July, 1941, (UK)]"


In [191]:
movie_df[movie_df['other_release_date'].notnull()]

,title,country,runtime_minutes,budget,global_gross,mpaa_rating,japan_release_date,usa_release_date,genres,imdb_user_rating,imdb_user_rating_count,oscar_wins,non_oscar_wins,metascore,is_Action,is_Adventure,is_Animation,is_Biography,is_Comedy,is_Crime,is_Drama,is_Family,is_Fantasy,is_History,is_Horror,is_Music,is_Musical,is_Mystery,is_News,is_Romance,is_Sci-Fi,is_Sport,is_Thriller,is_War,is_Western,other_release_date
1414,The Academy of Magic,USA,86.0,1.20e+07,NaN,None,NaT,5 January 2020 (Lebanon),"[Animation, Fantasy]",3.4,72.0,NaN,NaN,NaN,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5 January 2020 (Lebanon)
1471,Thomas & Friends: Digs & Discoveries,USA,66.0,NaN,NaN,None,NaT,11 September 2019 (Netherlands),"[Animation, Family]",5.0,6.0,NaN,NaN,NaN,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,11 September 2019 (Netherlands)
1487,Trouble,USA,87.0,NaN,1.08e+07,None,NaT,8 August 2019 (Thailand),"[Animation, Comedy, Family]",6.1,931.0,NaN,NaN,NaN,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8 August 2019 (Thailand)
1498,Outlander,USA,NaN,3.47e+00,NaN,None,NaT,4 July 2019 (Australia),[Animation],NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4 July 2019 (Australia)
1505,Norm of the North: King Sized Adventure,USA,90.0,NaN,1.44e+06,None,NaT,2 August 2019 (Vietnam),"[Animation, Adventure, Comedy, Family]",3.1,278.0,NaN,NaN,NaN,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2 August 2019 (Vietnam)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2786,The Wonderful World of the Brothers Grimm,USA,135.0,6.25e+06,NaN,G,None,16 July 1963 (UK),"[Animation, Adventure, Biography, Comedy, Dram...",6.4,1420.0,1.0,1.0,NaN,0,1,1,1,1,0,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,16 July 1963 (UK)
2788,The World of Topo Gigio,USA,75.0,NaN,NaN,None,None,1961 (Italy),"[Animation, Family, Fantasy]",5.6,40.0,NaN,1.0,NaN,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1961 (Italy)
2790,Donald Duck and his Companions,USA,NaN,NaN,NaN,None,None,13 September 1968 (Italy),"[Family, Animation, Comedy]",7.0,186.0,NaN,NaN,NaN,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,13 September 1968 (Italy)
2830,Fantasia,USA,125.0,2.28e+06,7.64e+07,G,None,21 July 1941 (UK),"[Animation, Family, Fantasy, Music]",7.8,86287.0,NaN,8.0,96.0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,21 July 1941 (UK)


In [183]:
movie_df['other_release_date'] = movie_df.apply(
    lambda x: x['usa_release_date'] if x['usa_release_date'].str.contains(r'\((.+)\)') else None, axis=1)

AttributeError: 'NoneType' object has no attribute 'str'

In [181]:
movie_df.iloc[[2834, 2830]]

,title,country,runtime_minutes,budget,global_gross,mpaa_rating,japan_release_date,usa_release_date,genres,imdb_user_rating,imdb_user_rating_count,oscar_wins,non_oscar_wins,metascore,is_Action,is_Adventure,is_Animation,is_Biography,is_Comedy,is_Crime,is_Drama,is_Family,is_Fantasy,is_History,is_Horror,is_Music,is_Musical,is_Mystery,is_News,is_Romance,is_Sci-Fi,is_Sport,is_Thriller,is_War,is_Western
2834,The Big Broadcast of 1938,USA,91.0,NaN,NaN,None,None,9 March 1938 (Sweden),"[Animation, Comedy, Musical, Romance]",6.4,838.0,1.0,1.0,NaN,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
2830,Fantasia,USA,125.0,2.28e+06,7.64e+07,G,None,21 July 1941 (UK),"[Animation, Family, Fantasy, Music]",7.8,86287.0,NaN,8.0,96.0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
